<a href="https://colab.research.google.com/github/untergeekDE/KInsKI-und-andere-spielereien/blob/main/KInsKIBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot
!pip install aleph-alpha-client
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


API-Token für den Bot und die Services einlesen (wie üblich als Textdatei im GDrive)

In [ ]:
import hashlib
from google.colab import drive

drive.mount('/content/drive')

keyfile = open('/content/drive/MyDrive/keys/kinski_bot.txt','r')
tg_bot_token = keyfile.readline()
keyfile.close()

keyfile = open('/content/drive/MyDrive/keys/aa_key.txt','r')
aa_token = keyfile.readline()
keyfile.close()

keyfile = open('/content/drive/MyDrive/keys/openai_key.txt','r')
openai_token = keyfile.readline()
keyfile.close()

print("AlephAlpha Token (MD5)", hashlib.md5(aa_token.encode('utf-8')).hexdigest(),"\n")
print("OpenAI Token (MD5)", hashlib.md5(openai_token.encode('utf-8')).hexdigest(),"\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
AlephAlpha Token (MD5) d7644b85dbe845bf36c85a166ee72f41 

OpenAI Token (MD5) 4dd938fcec2f1d51f9aa58083aa1d3b5 



Das Prompt einlesen: (Kurzbeschreibung und Frage-Antwort-Paare; endet auf eine Kinski-Antwort)



In [ ]:
# Lies die Kinski-Texte ein - ohne Frage
def load_kinski_prompt(): 
  promptfile = open('/content/drive/MyDrive/kinski_prompt.txt','r')
  promptlines = "".join(promptfile.readlines())
  promptfile.close()
  return(promptlines)

Hier die Antwortfunktion mit Aleph Alpha als Service:

In [ ]:
from aleph_alpha_client import AlephAlphaModel, AlephAlphaClient, CompletionRequest, Prompt
model = AlephAlphaModel(
    AlephAlphaClient(host="https://api.aleph-alpha.com", token=aa_token),
    model_name = "luminous-supreme"
)

def aleph_complete(prompt):
  request = CompletionRequest (
      prompt = Prompt(prompt),
      maximum_tokens = 256,
      temperature = 1.0,
      top_k = 10,
      top_p = .27,
      use_multiplicative_presence_penalty=True,
      presence_penalty=0.6,
      frequency_penalty=0.6,
      stop_sequences = ["###"],

    )
  result = model.complete(request)
  t = result.completions
  return(t[0].completion)

print("AA-Funktion definiert")

AA-Funktion definiert


OpenAI-Completion-Funktion

In [ ]:
import openai

openai.api_key = openai_token

def openai_complete(p):
  response = openai.Completion.create(
    model="text-davinci-002",
    prompt=p,
    temperature=1,
    max_tokens=256,
    top_p=0.27,
    best_of=3,
    frequency_penalty=0.6,
    presence_penalty=0.6,
    stop=["###"]
  )
  return(response.choices[0].text)

  #return(response)
print("OpenAI-Completion definiert")

OpenAI-Completion definiert


# The Bot

In [ ]:
# Globale Variablen
bot_version = "v0.1"
prompt = ""
n_fragen = 0
model = "aleph"

"""
Simple Bot to reply to Telegram messages.

First, a few handler functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.

Usage:
Basic Echobot example, repeats messages.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update, context):
    global prompt
    prompt = load_kinski_prompt()
    global n_fragen
    n_fragen = 0
    """Send a message when the command /start is issued."""
    update.message.reply_text('Herr Kinski ist jetzt da. Sie können Ihre Frage stellen.')


def help(update, context):
    global n_fragen
    global model
    """Send a message when the command /help is issued."""
    update.message.reply_text("### KInsKI Bot " + bot_version + " ###\n\n" \
                              + "Chatte mit Klaus Kinski - in einer KI-Variante: " \
                              + "Die KI-Sprachmodelle OpenAI GPT-3 und Aleph Alpha Luminous Extreme " \
                              + "antworten, wie Kinski geantwortet hätte. \n\n" \
                              + "Spätestens nach 10 Fragen ist Schluss. Ask wisely.\n\n" \
                              + "Spenden für die Modell-Kosten per Paypal an jan@eggers-elektronik.de\n\n" \
                              + "*Befehle:* \n" \
                              + "/help, /info - diese Hilfe\n" \
                              + "/start - Neues Gespräch starten\n" \
                              + "/openai - Schalte auf Modell GPT3\n" \
                              + "/aleph - Schalte auf Modell Luminous\n" \
                              + "Fragen gestellt: " + n_fragen + ", Modell: " + model)


def KI_wrapper(frage):
    global prompt
    global model
    p = prompt + "###\nFrage: " + frage + "\nKinski:"
    if model == "aleph":
      antwort = aleph_complete(p)
    else:
      antwort = openai_complete(p)
    return(antwort)

def answer(update, context):
    """Echo the user message."""
    global prompt
    global n_fragen
    if n_fragen < 10:
      antwort = KI_wrapper(update.message.text)
      update.message.reply_text('*' + antwort + '*')
      prompt = prompt + "###\nFrage: " + update.message.txt + "\nKinski: " + antwort 
      n_fragen += 1
      if n_fragen > 8: 
        update.message.reply_text('*Letzte Frage jetzt, sorry!*')
    else:
      update.message.reply_text('Herr Kinski musste wirklich gehen. \n\n' \
                                + 'Komm später wieder oder starte eine neue Session mit /start\n'
                                + 'Wenn dir KInsKI gefällt: Spenden für den Betrieb an die Paypal-' \
                                + 'Adresse jan@eggers-elektronik.de - weil jede Antwort ein paar ' \
                                + 'Cent in Rechenzeit kostet... danke!\n')


def set_aleph(update, context):
    """Switch to Aleph Alpha model."""
    model = "aleph"
    update.message.reply_text('Aktives Modell jetzt: Aleph Alpha Luminous (Supreme)\n')

def set_openai(update, context):
    """Switch to OpenAI model."""
    model = "openai"
    update.message.reply_text('Aktives Modell jetzt: OpenAI GPT-3 (Da Vinci)\n')

def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)

def main():
    """Starte KInsKI."""
    global prompt
    prompt = load_kinski_prompt()
    global n_fragen
    n_fragen = 0
    global model
    model = "aleph"


    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater(tg_bot_token, use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram)
    dp.add_handler(CommandHandler("info", help))
    dp.add_handler(CommandHandler("help", help))
    dp.add_handler(CommandHandler("openai", set_openai))
    dp.add_handler(CommandHandler("gpt3",set_openai))
    dp.add_handler(CommandHandler("aleph", set_aleph))
    dp.add_handler(CommandHandler("luminous",set_aleph))

    # on noncommand i.e message - answer on telegram
    dp.add_handler(MessageHandler(Filters.TEXT, answer))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()